In [1]:
from nltk import ngrams


text = "Tôi yêu Việt Nam"

words = text.split() + ["\x00"] * max(0, 5 - len(text.split()))

for ngram in ngrams(words, 5):
    print(" ".join(ngram))

Tôi yêu Việt Nam  


In [2]:
from core.models.accent_restore_model import AccentRestoreModel
from core.config.config import get_config
from core.database.database import Database

In [3]:
import os

database = Database(
    path_indomain=get_config("Path", "indomain"),
    path_outdomain=get_config("Path", "outdomain")
)
accent_model = AccentRestoreModel(
    path = os.path.join(get_config("Path", "model"), get_config("Model", "accent")),
)

02:28:02 [ INFO      ] CHATBOT_UIT:  Reading file
02:28:02 [ INFO      ] CHATBOT_UIT:  Reading share knowledge
02:28:02 [ INFO      ] CHATBOT_UIT:  Reading answer database
c:\ProgramData\miniconda3\envs\cb_uit\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
02:28:02 [ INFO      ] CHATBOT_UIT:  Reading product
c:\ProgramData\miniconda3\envs\cb_uit\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
02:28:03 [ INFO      ] CHATBOT_UIT:  Loading accent model from: Models\accent


True


ValueError: File not found: filepath=Models\accent\model.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
import numpy as np
from collections import Counter
import re

test = "diem chuan 2024"

ngrams = accent_model.process_phrase(test)
X = np.array([accent_model.encode(accent_model.remove_accent(ngram)) for ngram in ngrams])

preds = accent_model.model.predict(X)
    
preds = [accent_model.decode(pred) for pred in preds]

candidates = [Counter() for _ in range(len(preds) + min(accent_model.config["ngram"], len(preds[0].split())) - 1)]
for nid, ngram in enumerate(preds):
    for wid, word in enumerate(re.split(' +', ngram)):
        idx = nid + wid
        if idx < len(candidates):
            candidates[idx].update([word])

output = ' '.join(c.most_common(1)[0][0] for c in candidates)
output

1/1 [==============================] - 0s 81ms/step


'điểm chuẩn 2024'

In [ ]:


replace_digits(test, output)

'điểm chuẩn 2024'

In [ ]:
re.findall(r'\d+', test)

['2024']